# Initialization

Test notebook for the C-MAPPS benchmark. Approach using MLP. 

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.font_manager
import matplotlib
import seaborn as sns
import pandas as pd
import logging
import random
#import plottingTools
#from datetime import datetime
#from sklearn.covariance import EllipticEnvelope
#from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#from sklearn.dummy import DummyClassifier
#from sklearn.model_selection import train_test_split, cross_validate
#from sklearn.neural_network import MLPClassifier
#from mpl_toolkits.mplot3d import Axes3D
#from dataManagement import DataManagerDamadics
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout

from IPython.display import display, HTML
%matplotlib notebook

global constRUL

constRUL = 125
time_window = 30
rul_vector = None

Using TensorFlow backend.


In [2]:
def compute_training_RUL(df_row, *args):
    
    global constRUL
    rul_vector = args[0]
    
    if rul_vector[int(df_row['Unit Number']) - 1] - df_row['Cycle'] > constRUL:
        return constRUL
    else:
        return rul_vector[int(df_row['Unit Number']) - 1] - df_row['Cycle']

In [3]:
def get_X_y_from_df(df, time_window, features, num_units, dataset_type):
    
    n_m = df.shape[0]
    n_x = len(features)
    
    df_values = df[features].values
    targets = df['RUL'].values
    n_m = 0
    n_X = len(features)
    df_unit_values = []
    targets_unit = []
    num_samples_unit = []
    
    #Count number of elements at each group so that we can create the matrix to hold them all. 
    #Also store each matrix in temporary arrays to access them faster
    for i in range(1,num_units+1):
        
        df_unit = df.loc[df['Unit Number'] == i]
        df_unit_values.append(df_unit[features].values) #is this a view or a copy of the df?
        targets_unit.append(df_unit['RUL'].values) #is this a view or a copy of the df?
        num_samples_unit.append(df_unit.shape[0])
        n_m = n_m + num_samples_unit[i-1]-time_window+1
    
    #Create the numpy arrays to hold the features
    if (dataset_type == 'train' or dataset_type == 'cross_validation'):
        X, y = np.empty([n_m, n_x*time_window]), np.empty([n_m, 1])
    else:
        X, y = np.empty([num_units, n_x*time_window]), np.empty([num_units, 1])
        
    k = 0
    
    #Create the feature matrix by moving the time window for each type of engine.
    for i in range(num_units):
    
        if (dataset_type == 'train' or dataset_type == 'cross_validation'):
            for j in range(num_samples_unit[i]-time_window+1):

                time_window_samples = df_unit_values[i][j:j+time_window]
                X[k,:] = np.squeeze(time_window_samples.reshape(1,-1))
                y[k] = targets_unit[i][j+time_window-1]
                k = k + 1
        else:
            #print(dataset_type)
            time_window_samples = df_unit_values[i][-time_window:]
            X[k,:] = np.squeeze(time_window_samples.reshape(1,-1))
            k = k + 1
    
    return X, y

# Retrieve and Reshape data

Get the data from the text files, store it in a Pandas Dataframe and reshape it as appropiately.

In [4]:
def retrieve_and_reshape_data(from_file, columns, time_window, dataset_type):
    '''
    T2        - Total temperature at fan inlet      R
    T24       - Total temperature at lpc outlet     R
    T30       - Total temperature at hpc outlet     R
    T50       - Total temperature at LPT outlet     R
    P2        - Pressure at fan inlet               psia
    P15       - Total pressure in bypass-duct       psia
    P30       - Total pressure at HPC outlet        psia
    Nf        - Physical fan speed                  rpm
    Nc        - Physical core speed                 rpm
    epr       - Engine Pressure ratio (P50/P2)      --
    Ps30      - Static Pressure at HPC outlet       psia
    phi       - Ratio fuel flow to Ps30             pps/psi
    NRf       - corrected fan speed                 rpm
    NRc       - Corrected core speed                rpm
    BPR       - Bypass ratio                        --
    farB      - Burner fuel-air ratio               --
    htBleed   - Bleed enthalpy                      --
    Nf_dmd    - Demanded fan speed                  rpm
    PCNfR_dmd - Demanded corrected fan speed        rpm
    W31       - HPT coolant bleed                   lbm/s
    W32       - LPT coolant bleed                   lbm/s
    '''

    
    df = pd.read_csv(from_file ,sep='\s+',header=None)

    col_names = {0:'Unit Number', 1:'Cycle', 2:'Op. Settings 1', 3:'Op. Settings 2', 4:'Op. Settings 3', 5:'T2',
                6:'T24', 7:'T30', 8:'T50', 9:'P2', 10:'P15', 11:'P30', 12:'Nf', 13:'Nc', 14:'epr', 15:'Ps30', 
                16:'phi', 17:'NRf', 18:'NRc', 19:'BPR', 20:'farB', 21:'htBleed', 22:'Nf_dmd', 23:'PCNfR_dmd', 
                24:'W31', 25:'W32'}

    df.rename(columns=col_names, inplace=True)

    gruoped_by_unit = df.groupby('Unit Number')
    rul_vector = gruoped_by_unit.size().values
    num_units = len(gruoped_by_unit)

    df['RUL'] = df.apply(compute_training_RUL, axis = 1, args=(rul_vector,))
    selected_features_rul = columns[:]
    selected_features_rul.extend(['Unit Number', 'RUL'])
    df_selected_features = df[selected_features_rul]

    X, y = get_X_y_from_df(df_selected_features, time_window, selected_features, num_units, dataset_type)
    
    #display(df_selected_features.head(5))
    
    return X, y

    '''display(df_selected_features)
    print(X.shape)
    print(X)
    print(y.shape)
    print(y)'''

# Keras model

We will use a very simple ANN for this example. The model is Dense(ReLU, 100)->Dense(ReLu, 100)->Dense(Linear, 1)

In [14]:
def RULmodel(input_shape):
    
    print(input_shape)
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(1000, input_dim=input_shape, activation='relu', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(500, activation='relu', name='fc2'))
    model.add(Dropout(0.5))
    #model.add(Dense(100, activation='relu', name='fc3'))
    #model.add(Dropout(0.5))
    #model.add(Dense(10, activation='relu', name='fc4'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear', name='out'))
    
    #create a placeholder for the input
    #X_input = Input(shape=(input_shape))
    
    #Create the layers
    #X = Dense(100, activation='relu', name='fc1')(X_input)
    #X = Dense(100, activation='relu', name='fc2')(X)
    #X = Dense(1, activation='linear', name='out')(X)
    
    # Create model. This creates the Keras model instance, you'll use this instance to train/test the model.
    #model = Sequential(inputs = X_input, outputs = X, name='RUL')
    
    return model
    
    

# Retrieve the train and test data from the files

In [6]:
data_file_train = '../CMAPSSData/train_FD001.txt'
data_file_test = '../CMAPSSData/test_FD001.txt'

#min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
standardScaler = StandardScaler()

#Selected as per CNN paper
selected_features = ['T2', 'T24', 'T30', 'P15', 'P30', 'Nf', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'farB', 'PCNfR_dmd', 'W31']

#Get the X and y matrices with the specified time window
X_train, y_train = retrieve_and_reshape_data(data_file_train, selected_features, time_window, 'train')
X_test, _ = retrieve_and_reshape_data(data_file_test, selected_features, time_window, 'test')
y_test = np.loadtxt("../CMAPSSData/RUL_FD001.txt")
y_test = np.reshape(y_test, (y_test.shape[0], 1))

#Standardize the data
X_train = standardScaler.fit_transform(X_train)
X_test = standardScaler.fit_transform(X_test)



In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_train[-5:,:])
print(y_train[-5:,:])
print(X_test[-5:,:])
print(y_test[-5:,:])

(17731, 420)
(17731, 1)
(100, 420)
(100, 1)
[[ 1.          2.41264288  1.49234946 ...,  1.          0.         -1.69899007]
 [ 1.          0.97293491  0.82670323 ...,  1.          0.         -2.7492681 ]
 [ 1.          0.68499331  0.82670323 ...,  1.          0.         -1.97537902]
 [ 1.          0.34906145  0.82670323 ...,  1.          0.         -2.80454589]
 [ 1.          1.16489597  0.85379348 ...,  1.          0.         -2.36232356]]
[[ 4.]
 [ 3.]
 [ 2.]
 [ 1.]
 [ 0.]]
[[ 1.         -1.54886617 -1.88125364 ...,  1.          0.          1.25789073]
 [ 1.          0.59455611 -0.54071589 ...,  1.          0.         -0.99754859]
 [ 1.          0.05870054  0.58060223 ...,  1.          0.         -0.03093174]
 [ 1.         -0.96429646 -0.00915678 ...,  1.          0.          1.1934496 ]
 [ 1.         -0.55022624  0.30900268 ...,  1.          0.         -0.41757848]]
[[ 137.]
 [  82.]
 [  59.]
 [ 117.]
 [  20.]]


# Fit the keras model

Fit the Keras model to the data and determine its performance.

In [15]:
#Create the model
modelRUL = RULmodel(X_train.shape[1])

#Compile the model.
modelRUL.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["accuracy"])

#Train the model.
modelRUL.fit(x = X_train, y = y_train, epochs = 200, batch_size = 128)

#Evaluate the model
preds = modelRUL.evaluate(x = X_test, y = y_test)

print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

420
Epoch 1/200
17731/17731 [==============================] - 5s 301us/step - loss: 719.8961 - acc: 0.0235
Epoch 2/200
17731/17731 [==============================] - 5s 277us/step - loss: 281.2681 - acc: 0.0324
Epoch 3/200
17731/17731 [==============================] - 5s 277us/step - loss: 266.1248 - acc: 0.0327 0s - loss: 265.8550 
Epoch 4/200
17731/17731 [==============================] - 5s 284us/step - loss: 265.3448 - acc: 0.0339
Epoch 5/200
17731/17731 [==============================] - 5s 274us/step - loss: 263.0590 - acc: 0.0350
Epoch 6/200
17731/17731 [==============================] - 5s 281us/step - loss: 261.5475 - acc: 0.0343
Epoch 7/200
17731/17731 [==============================] - 5s 279us/step - loss: 251.5504 - acc: 0.0322
Epoch 8/200
17731/17731 [==============================] - 5s 301us/step - loss: 240.4054 - acc: 0.0345
Epoch 9/200
17731/17731 [==============================] - 5s 278us/step - loss: 237.1390 - acc: 0.0370
Epoch 10/200
17731/17731 [=============

KeyboardInterrupt: 